In [ ]:
import os
import sys
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

import cv2
from PIL import Image as im
from sklearn.metrics import jaccard_score

import collections
from typing import DefaultDict, Tuple, List, Dict
from functools import partial

In [ ]:
class VAE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.input_shape = kwargs["input_shape"]
        # number of hidden units in first hidden layer
        self.n_units = kwargs["n_units"]
        # number of hidden units in latent space
        self.latent_units = kwargs["latent_units"]
        
        self.encoder = nn.Sequentual(
                            nn.Conv2d(
                                in_channels=1, 
                                out_channels=64,
                                kernel_size=3, 
                                stride=2, 
                                padding=1),
                            nn.BatchNorm2d(64),
                            nn.ReLU(),
                            nn.Conv2d(
                                in_channels=64, 
                                out_channels=64,
                                kernel_size=3, 
                                stride=1, 
                                padding=1),
                            nn.BatchNorm2d(64),
                            nn.ReLU())
        
        self.mean = nn.Linear(256, 4)
        self.log_var = nn.Linear(256, 4)
        
        self.decoder_input = nn.Linear(4, 256)
        
        self.decoder = nn.Sequential(
                            nn.ConvTranspose2d(
                                hidden_dims[i],
                                hidden_dims[i + 1],
                                kernel_size=3,
                                stride = 2,
                                padding=1,
                                output_padding=1),
                            nn.BatchNorm2d(hidden_dims[i + 1]),
                            nn.ReLU())
        
        self.final_layer = nn.Sequential(
                                nn.ConvTranspose2d(
                                    hidden_dims[-1],
                                    hidden_dims[-1],
                                    kernel_size=3,
                                    stride=2,
                                    padding=1,
                                    output_padding=1),
                                nn.BatchNorm2d(hidden_dims[-1]),
                                nn.LeakyReLU(),
                                nn.Conv2d(
                                    hidden_dims[-1], 
                                    out_channels=3,
                                    kernel_size=3, 
                                    padding=1),
                                nn.Sigmoid())
    def forward(self, X):
        return